# CLI Workflows - Advanced Usage Patterns

This notebook demonstrates advanced command-line workflows and usage patterns for pip-project-template. You'll learn:

1. 🔄 Chaining CLI commands together
2. 📊 Batch processing operations  
3. 🛡️ Error handling and validation
4. 🖥️ Server management workflows
5. 🔗 Integration with other tools
6. ⚡ Advanced automation patterns

In [ ]:
# Setup and imports
import subprocess
import sys
import json
import time
import concurrent.futures
from pathlib import Path
from contextlib import contextmanager
from io import StringIO

# Helper function to run CLI commands
def run_cli_command(module, args=None, capture=True):
    """Helper to run CLI commands and capture output."""
    args = args or []
    cmd = [sys.executable, "-m", f"pip_project_template.cli.{module}"] + args
    
    if capture:
        result = subprocess.run(cmd, capture_output=True, text=True)
        return result.returncode, result.stdout, result.stderr
    else:
        return subprocess.run(cmd)

print("🔧 CLI Workflows setup complete!")

## 1. Basic CLI Workflows

Let's start with fundamental CLI operations:

In [ ]:
# Get system information
print("📋 Getting system information...")
returncode, stdout, stderr = run_cli_command("info")

if returncode == 0:
    print("✅ Info command successful:")
    print(stdout)
else:
    print(f"❌ Info command failed: {stderr}")

In [ ]:
# Basic calculations
print("🧮 Running basic calculations:\n")

test_calculations = [
    (["10", "5", "--operation", "add"], "10 + 5"),
    (["20", "4", "--operation", "multiply"], "20 × 4"),
    (["15.5", "2.5", "--operation", "add"], "15.5 + 2.5"),
]

results = []
for args, description in test_calculations:
    returncode, stdout, stderr = run_cli_command("calculate", args)
    if returncode == 0:
        results.append((description, stdout.strip()))
        print(f"✅ {description}: {stdout.strip()}")
    else:
        print(f"❌ {description} failed: {stderr}")

print(f"\n📊 Completed {len(results)} calculations successfully")

## 2. Batch Processing

Process multiple operations efficiently:

In [ ]:
# Batch processing example
print("📦 Batch processing multiple calculations:\n")

calculations = [
    (100, 25, "add"),
    (50, 3, "multiply"), 
    (200, 8, "add"),
    (12, 7, "multiply"),
    (75, 15, "add")
]

batch_results = []
for i, (a, b, op) in enumerate(calculations, 1):
    returncode, stdout, stderr = run_cli_command(
        "calculate", 
        [str(a), str(b), "--operation", op]
    )
    
    if returncode == 0:
        result = stdout.strip()
        batch_results.append((a, b, op, result))
        print(f"✅ Batch {i}: {a} {op} {b} → {result}")
    else:
        print(f"❌ Batch {i}: {a} {op} {b} failed: {stderr}")

print(f"\n🎯 Batch processing complete: {len(batch_results)}/{len(calculations)} successful")

In [ ]:
# Process results into a summary
try:
    import pandas as pd
    HAS_PANDAS = True
except ImportError:
    HAS_PANDAS = False

if batch_results:
    # Create a summary table
    summary_data = []
    for a, b, op, result_str in batch_results:
        # Extract numeric result from string like "100.0 add 25.0 = 125.0"
        try:
            result_value = float(result_str.split("=")[-1].strip())
            summary_data.append({
                'Input A': a,
                'Operation': op,
                'Input B': b,
                'Result': result_value
            })
        except (ValueError, IndexError):
            print(f"⚠️ Could not parse result: {result_str}")
    
    if summary_data and HAS_PANDAS:
        df = pd.DataFrame(summary_data)
        print("\n📊 Batch Processing Summary:")
        print(df.to_string(index=False))
        
        # Basic statistics
        print(f"\n📈 Statistics:")
        print(f"   Total operations: {len(df)}")
        print(f"   Add operations: {len(df[df['Operation'] == 'add'])}")
        print(f"   Multiply operations: {len(df[df['Operation'] == 'multiply'])}")
        print(f"   Average result: {df['Result'].mean():.2f}")
        print(f"   Max result: {df['Result'].max():.2f}")
        print(f"   Min result: {df['Result'].min():.2f}")
    elif summary_data:
        print("\n📊 Summary (without pandas):")
        for item in summary_data:
            print(f"   {item['Input A']} {item['Operation']} {item['Input B']} = {item['Result']}")
else:
    print("⚠️ No results to summarize")

## 3. Error Handling and Validation

Test how the CLI handles various error conditions:

In [ ]:
# Test error handling
print("🛡️ Testing error handling:\n")

error_cases = [
    (["abc", "def", "--operation", "add"], "Non-numeric inputs"),
    (["10", "5", "--operation", "invalid"], "Invalid operation"),
    (["10"], "Missing required argument"),
    (["10", "5", "6", "--operation", "add"], "Too many arguments"),
]

error_summary = []
for args, description in error_cases:
    returncode, stdout, stderr = run_cli_command("calculate", args)
    
    if returncode != 0:
        error_summary.append((description, "Properly handled", stderr.strip()))
        print(f"✅ {description}: Properly handled")
        print(f"   Error message: {stderr.strip()}")
    else:
        error_summary.append((description, "Should have failed", stdout.strip()))
        print(f"❌ {description}: Should have failed but returned: {stdout.strip()}")
    print()

print(f"📋 Error handling summary: {len([e for e in error_summary if e[1] == 'Properly handled'])}/{len(error_cases)} handled correctly")

## 4. Server Management Workflows

Learn how to work with MCP servers:

In [ ]:
# Check MCP configuration
print("🔧 Checking MCP server configuration:\n")

config_file = Path("config/mcp_config.json")
if config_file.exists():
    try:
        with open(config_file) as f:
            config = json.load(f)
        
        servers = config.get("mcpServers", {})
        print(f"✅ Found {len(servers)} configured MCP servers:")
        print()
        
        server_info = []
        for server_name, server_config in servers.items():
            if "command" in server_config:
                transport = "Command-based (stdio)"
                endpoint = server_config["command"]
            elif "url" in server_config:
                transport = f"HTTP-based"
                endpoint = server_config["url"]
            else:
                transport = "Unknown transport"
                endpoint = "N/A"
            
            server_info.append({
                'Server': server_name,
                'Transport': transport,
                'Endpoint': endpoint
            })
            
            print(f"📡 **{server_name}**")
            print(f"   Transport: {transport}")
            print(f"   Endpoint: {endpoint}")
            
            if "env" in server_config:
                env_vars = list(server_config["env"].keys())
                print(f"   Environment: {', '.join(env_vars)}")
            print()
        
        # Create summary table
        if server_info and HAS_PANDAS:
            df_servers = pd.DataFrame(server_info)
            print("📊 Server Configuration Summary:")
            print(df_servers.to_string(index=False))
            
    except json.JSONDecodeError:
        print("❌ MCP config file exists but contains invalid JSON")
    except Exception as e:
        print(f"❌ Error reading MCP config: {e}")
else:
    print("❌ MCP config file not found at config/mcp_config.json")

## 5. Advanced Workflow Patterns

Complex automation and chaining examples:

In [ ]:
# Parallel processing simulation
print("⚡ Parallel Processing Simulation:\n")

def timed_calculation(args):
    """Perform a calculation and measure time."""
    a, b, op = args
    start_time = time.time()
    
    returncode, stdout, stderr = run_cli_command(
        "calculate", 
        [str(a), str(b), "--operation", op]
    )
    
    end_time = time.time()
    duration = end_time - start_time
    
    if returncode == 0:
        return args, stdout.strip(), duration, None
    else:
        return args, None, duration, stderr

# Set of calculations to run in parallel
parallel_tasks = [
    (100, 50, "add"),
    (75, 25, "multiply"),
    (200, 35, "add"),
    (15, 8, "multiply"),
]

print(f"🔄 Running {len(parallel_tasks)} calculations in parallel...\n")

# Sequential execution for comparison
start_time = time.time()
sequential_results = [timed_calculation(task) for task in parallel_tasks]
sequential_time = time.time() - start_time

# Parallel execution
start_time = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    parallel_results = list(executor.map(timed_calculation, parallel_tasks))
parallel_time = time.time() - start_time

# Analyze results
print("📊 Performance Comparison:")
print(f"   Sequential execution: {sequential_time:.3f} seconds")
print(f"   Parallel execution: {parallel_time:.3f} seconds")
print(f"   Speedup: {sequential_time/parallel_time:.2f}x")

print("\n✅ Parallel Results:")
for (a, b, op), result, duration, error in parallel_results:
    if error is None:
        print(f"   {a} {op} {b} = {result} ({duration:.3f}s)")
    else:
        print(f"   ❌ {a} {op} {b} failed: {error} ({duration:.3f}s)")

## 🎯 Workflow Best Practices

Based on the examples above, here are key best practices for CLI workflows:

### ✅ **Do's:**
- Always check return codes for error handling
- Use structured output parsing for reliable automation
- Implement retry logic for network operations
- Log all operations for debugging
- Use configuration files for repeated setups
- Validate inputs before processing
- Consider parallel processing for independent operations

### ❌ **Don'ts:**
- Don't ignore error conditions
- Don't hardcode values that might change
- Don't assume operations will always succeed
- Don't forget to clean up resources
- Don't skip input validation

### 🛠️ **Tools Integration:**
- **Shell scripts**: Great for system automation
- **Python scripts**: Best for complex logic and data processing  
- **CI/CD pipelines**: Automate testing and deployment
- **Process managers**: Handle server lifecycle in production
- **Monitoring**: Track server health and performance

## Next Steps

- **mcp_integration.ipynb** - Learn multi-server coordination
- **extending/** - Create custom commands and servers
- Try implementing your own workflow automation
- Experiment with different integration patterns